In [1]:
# Load packages

import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import json

In [2]:
# Urban rural trends over time

pop_data = pd.read_csv('UrbanRural.csv')
# Total population
total = pop_data[pop_data['Area'] == 'Total']
fig = px.bar(total, x='Group', y='Total Rate',
             color='Year', barmode='group', width=750, title='Childhood mortality rates',
             color_discrete_sequence=['#006400', '#228B22', '#32CD32', '#66CDAA','#8FBC8F']
             )
fig.update_layout(
    yaxis_title='Deaths per 100 live births'
)
fig.show()


In [3]:
# Urban vs rural
UR = pop_data[pop_data['Area'] != 'Total']
# Neonatal mortality
UR_neonatal = UR[UR['Group'] == 'Neonatal mortality']
fig = px.bar(UR_neonatal, x='Year', y='Total Rate',
             color='Area', barmode='group', width=500,
             title='Neonatal mortality rates',
             color_discrete_sequence=['#CC6600', '#FFB266']
             )
# Update layout
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
)
fig.show()
# Postneonatal mortality
UR_postnatal = UR[UR['Group'] == 'Postneonatal mortality']
fig = px.bar(UR_postnatal, x='Year', y='Total Rate',
             color='Area', barmode='group', width=500, title='Postneonatal mortality rates',
             color_discrete_sequence=['#CC6600', '#FFB266']
             )
# Update layout
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
)
fig.show()
# Child mortality
UR_child = UR[UR['Group'] == 'Child mortality']
fig = px.bar(UR_child, x='Year', y='Total Rate',
             color='Area', barmode='group', width=500, title='Child mortality rates',
             color_discrete_sequence=['#CC6600', '#FFB266']
             )
# Update layout
fig.update_layout(
    yaxis_title='Deaths per 100 live births'
    )
fig.show()

# Under five mortality
UR_underfive = UR[UR['Group'] == 'Under five mortality']
fig = px.bar(UR_underfive, x='Year', y='Total Rate',
             color='Area', barmode='group', width=500, title='Under five mortality rates',
             color_discrete_sequence=['#CC6600', '#FFB266']
             )
# Update layout
fig.update_layout(
    yaxis_title='Deaths per 100 live births'
    )
fig.show()


In [4]:
# Proportion of rural to urban mortality rates
def proportion(data):
    temp = data.pivot(index='Year', columns='Area', values='Total Rate')
    temp['Proportion'] = temp['Rural'] / temp['Urban']
    data_p = temp.reset_index()
    return data_p

UR_neonatal_p = proportion(UR_neonatal)
UR_postnatal_p = proportion(UR_postnatal)
UR_child_p = proportion(UR_child)
UR_underfive_p = proportion(UR_underfive)

# Join into one dataframe
UR_neonatal_p['Group'] = 'Neonatal mortality'
UR_postnatal_p['Group'] = 'Postneonatal mortality'
UR_child_p['Group'] = 'Child mortality'
UR_underfive_p['Group'] = 'Under five mortality'
UR_p = pd.concat([UR_neonatal_p, UR_postnatal_p, UR_child_p, UR_underfive_p])

# Plot proportion
fig = px.bar(UR_p, x='Group', y='Proportion',
             color='Year', barmode='group', width=750,
             color_discrete_sequence=['#996515', '#B38600', '#CC9900', '#E5BF33', '#F2D666']
             )
# Update layout
fig.update_layout(
    yaxis_title='Ratio of rural to urban mortality rates',
    yaxis=dict(range=[1, 2])  # Set y-axis range
)
fig.show()


In [5]:
# Load the CSV with GeoJSON geometries
gdf = gpd.read_file('geojson_filtered.geojson')
geojson_data = gdf.to_json()

# Load the CSV with population data
data = pd.read_csv('ICM_State.csv')

neonatal = data[data['Group'] == 'Neonatal mortality'].reset_index(drop=True)
postneonatal = data[data['Group'] == 'Postneonatal mortality'].reset_index(drop=True)
child = data[data['Group'] == 'Child mortality'].reset_index(drop=True)
underFive = data[data['Group'] == 'Under five mortality'].reset_index(drop=True)

# Merge data back to one dataframe
data_all = {
    'NAME_1': neonatal['NAME_1'],
    'Year': neonatal['Year'],
    'Neonatal': neonatal['Rate'],
    'Postneonatal': postneonatal['Rate'],
    'Child': child['Rate'],
    'Under five': underFive['Rate']
}
data_all = pd.DataFrame(data_all)

merged_data = pd.merge(gdf, data_all, on='NAME_1')
merged_data.rename(columns={'NAME_1': 'State'}, inplace=True)

In [6]:
data = merged_data

current_year = '1992-1993'
current_mortality_rate = 'Neonatal'
years = sorted(data['Year'].unique())

fig = go.Figure()
# Add traces for each combination of year and mortality rate type
for year in years:
    visible = (year == current_year)
    fig.add_trace(go.Choropleth(
        locationmode='geojson-id',
        geojson=json.loads(geojson_data),
        featureidkey="properties.NAME_1",
        locations=data[data['Year'] == year]['State'].reset_index(drop=True),
        z=data[data['Year'] == year][current_mortality_rate].reset_index(drop=True),
        colorscale='Blues',
        colorbar_title="Rate /100",
        visible=visible,
        #name=f"{current_mortality_rate}, {year}"
    ))

# Dropdown menus to select the year and mortality rate
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="1992-1993",
                        method="update",
                        args=[
                            {"visible": [year == '1992-1993' for year in years]}]),
                dict(label="1998-1999",
                        method="update",
                        args=[
                            {"visible": [year == '1998-1999' for year in years]}]),
                dict(label="2005-2006",
                        method="update",
                        args=[
                            {"visible": [year == '2005-2006' for year in years]}]),
                dict(label="2015-2016",
                        method="update",
                        args=[{"visible": [year == '2015-2016' for year in years]},]),
                dict(label="2019-2021",
                        method="update",
                        args=[{"visible": [year == '2019-2021' for year in years]},])
            ]),
            type="buttons",
            direction="right",
            x=0.08,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        dict(
            buttons=[
                dict(label="Neonatal Mortality",
                        method="update",
                        args=[
                            {"z": [data[data['Year'] == year]['Neonatal'] for year in data['Year'].unique()]}]),
                dict(label="Postneonatal Mortality",
                        method="update",
                        args=[
                            {"z": [data[data['Year'] == year]['Postneonatal'] for year in data['Year'].unique()]}]),
                dict(label="Child Mortality",
                        method="update",
                        args=[
                            {"z": [data[data['Year'] == year]['Child'] for year in data['Year'].unique()]}]),
                dict(label="Under five Mortality",
                        method="update",
                        args=[
                            {"z": [data[data['Year'] == year]['Under five'] for year in data['Year'].unique()]}])
            ],
            direction="down",
            x=0.32,
            xanchor="left",
            y=1.3,
            yanchor="top"
        )
    ],
)

fig.update_geos(fitbounds="locations", 
                visible=True)
fig.show(config={
    'staticPlot': False,
    'scrollZoom': False,
    'doubleClick': False
})


In [7]:
total = pop_data[pop_data['Area'] == 'Total']
df = total[total['Group'] == 'Under five mortality']
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Male'],
    name='Male',
    marker_color='#CC79A7'
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Female'],
    name='Female',
    marker_color='#009E73'
))
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
    title='Under five mortality rates by sex'
    )
fig.show()


df = UR[UR['Group'] == 'Neonatal mortality']
marker = ['solid', 'dot']
fig = go.Figure()
count = 0
for area in df['Area'].unique():
    temp = df[df['Area'] == area]
    fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Male'],
                             marker=dict(size=10),
                             line=dict(color='#CC79A7', width=2, dash=marker[count]),
                             name=f"{area} - Male"))

    fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Female'],
                             marker=dict(size=10),
                             line=dict(color='#009E73', width=2, dash=marker[count]),
                             name=f"{area} - Female"))
    count += 1

fig.update_layout(
    title='Neonatal mortality rates',
    xaxis_title=None,
    yaxis_title='Deaths per 100 live births',
)
fig.show()

df = UR[UR['Group'] == 'Postneonatal mortality']
fig = go.Figure()
count = 0
for area in df['Area'].unique():
    temp = df[df['Area'] == area]
    fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Male'],
                             marker=dict(size=10),
                             line=dict(color='#CC79A7', width=2, dash=marker[count]),
                             name=f"{area} - Male"))

    fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Female'],
                             marker=dict(size=10),
                             line=dict(color='#009E73', width=2, dash=marker[count]),
                             name=f"{area} - Female"))
    count += 1

fig.update_layout(
    title='Postneonatal mortality rates',
    xaxis_title=None,
    yaxis_title='Deaths per 100 live births',
)
fig.show()


df = UR[UR['Group'] == 'Child mortality']
fig = go.Figure()
count = 0
for area in df['Area'].unique():
    temp = df[df['Area'] == area]
    fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Male'],
                             marker=dict(size=10),
                             line=dict(color='#CC79A7', width=2, dash=marker[count]),
                             name=f"{area} - Male"))

    fig.add_trace(go.Scatter(x=temp['Year'], y=temp['Female'],
                             marker=dict(size=10),
                             line=dict(color='#009E73', width=2, dash=marker[count]),
                             name=f"{area} - Female"))
    count += 1

fig.update_layout(
    title='Child mortality rates',
    xaxis_title=None,
    yaxis_title='Deaths per 100 live births',
)
fig.show()



In [8]:
# Mortality rates based on size at birth

total = pop_data[pop_data['Area'] == 'Total']
color_discrete_sequence=['#006D5B', '#00A79D', '#7ED5CF']

df = total[total['Group'] == 'Neonatal mortality']
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Very Small'],
    name='Very small',
    marker_color=color_discrete_sequence[0]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Small'],
    name='Small',
    marker_color=color_discrete_sequence[1]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Average or larger'],
    name='Average or larger',
    marker_color=color_discrete_sequence[2]
))
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
    title='Neonatal mortality rates by baby size at birth'
    )
fig.show()

df = total[total['Group'] == 'Postneonatal mortality']
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Very Small'],
    name='Very small',
    marker_color=color_discrete_sequence[0]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Small'],
    name='Small',
    marker_color=color_discrete_sequence[1]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Average or larger'],
    name='Average or larger',
    marker_color=color_discrete_sequence[2]
))
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
    title='Postneonatal mortality rates by baby size at birth'
    )
fig.show()


In [9]:
# Mortality rates based on gap between births

total = pop_data[pop_data['Area'] == 'Total']
color_discrete_sequence=['#A05282', '#C48BAF', '#D7AFC6']

df = total[total['Group'] == 'Neonatal mortality']
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Gap under 2'],
    name='Gap under 2',
    marker_color=color_discrete_sequence[0]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Gap 2-4'],
    name='Gap 2-4',
    marker_color=color_discrete_sequence[1]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Gap over 4'],
    name='Gap over 4',
    marker_color=color_discrete_sequence[2]
))
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
    title='Neonatal mortality rates by gap between births'
    )
fig.show()

df = total[total['Group'] == 'Child mortality']
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Gap under 2'],
    name='Gap under 2',
    marker_color=color_discrete_sequence[0]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Gap 2-4'],
    name='Gap 2-4',
    marker_color=color_discrete_sequence[1]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Gap over 4'],
    name='Gap over 4',
    marker_color=color_discrete_sequence[2]
))
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
    title='Child mortality rates by gap between births'
    )
fig.show()


In [10]:
total = pop_data[pop_data['Area'] == 'Total']
color_discrete_sequence=['#A05282', '#B2679A', '#C48BAF', '#D7AFC6']

df = total[total['Group'] == 'Neonatal mortality']
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Mother under 20'],
    name='Mother under 20',
    marker_color=color_discrete_sequence[0]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Mother 20-29'],
    name='Mother 20-29',
    marker_color=color_discrete_sequence[1]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Mother 30-39'],
    name='Mother 30-39',
    marker_color=color_discrete_sequence[2]
))
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Mother 40-49'],
    name='Mother 40-49',
    marker_color=color_discrete_sequence[3]
))
fig.update_layout(
    yaxis_title='Deaths per 100 live births',
    title='Neonatal mortality rates by age of mother at birth'
    )
fig.show()